In [ ]:
import os
import json
import pandas as pd
from collections import defaultdict


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 이미지 및 JSON 폴더 경로 설정
training_image_dir = '/content/drive/MyDrive/DCC/dataset/training_image'
validation_image_dir = '/content/drive/MyDrive/DCC/dataset/validation_image'
training_label_dir = '/content/drive/MyDrive/DCC/dataset/training_label'
validation_label_dir = '/content/drive/MyDrive/DCC/dataset/validation_label'


In [ ]:
# 2-1

# 이미지 파일 이름에서 이미지ID 추출하는 함수 
def extract_image_id(filename):
    return filename.split('_')[1]  # W_95569_19_normcore_W.jpg -> '95569'

In [ ]:
# 이미지ID 수집
def collect_image_ids(image_dir):
    image_ids = set()
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg'):
            image_id = extract_image_id(filename)
            image_ids.add(image_id)
    return image_ids


In [ ]:
# 이미지ID 수집
training_image_ids = collect_image_ids(training_image_dir)
validation_image_ids = collect_image_ids(validation_image_dir)


In [ ]:
# JSON 파일에서 성별, 스타일 정보를 추출
def extract_gender_style_from_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        gender = data['item']['gender']
        style = data['item']['style']
        return gender, style


In [ ]:
# 유효한 JSON 파일만 처리하여 통계 계산
def calculate_statistics(image_ids, label_dir):
    stats = defaultdict(lambda: defaultdict(int))

    for filename in os.listdir(label_dir):
        if filename.endswith('.json'):
            image_id = extract_image_id(filename)
            if image_id in image_ids:  # 이미지ID가 실제로 존재하는지 확인
                json_path = os.path.join(label_dir, filename)
                gender, style = extract_gender_style_from_json(json_path)
                stats[gender][style] += 1  # 성별 및 스타일에 따른 이미지 수 증가

    return stats


In [9]:
# 통계 계산
training_stats = calculate_statistics(training_image_ids, training_label_dir)
validation_stats = calculate_statistics(validation_image_ids, validation_label_dir)


OSError: [Errno 5] Input/output error: '/content/drive/MyDrive/DCC/dataset/training_label'

In [ ]:
# 통계 결과를 표 형식으로 변환하고 내림차순으로 정렬
def generate_table(stats, dataset_type):
    rows = []
    for gender, styles in stats.items():
        for style, count in styles.items():
            rows.append([gender, style, count])

    df = pd.DataFrame(rows, columns=['성별', '스타일', '이미지 수'])

    # 성별 기준으로 먼저 정렬한 후, 이미지 수 기준으로 내림차순 정렬
    df_sorted = df.sort_values(by=['성별', '이미지 수'], ascending=[True, False]).reset_index(drop=True)

    return df_sorted.head(1000)

In [ ]:
# 정렬된 표 작성
generate_table(training_stats, "Training")

In [ ]:
generate_table(validation_stats, "Validation")

In [ ]:
# 2-2

def collect_preference_data(image_ids, label_dir):
# JSON 파일에서 응답자 스타일 선호/비선호 정보를 수집하는 함수
# 매개변수설명 : image_ids 유효한 이미지 ID목록, label_dir JSON 파일이 위치한 디렉토리 경로
    preference_data = defaultdict(lambda: {'선호': [], '비선호': []})
    # defaultdict를 사용해 딕셔너리를 초기화하며, 응답자별로 선호/비선호 키를 가지는 딕셔너리 구조가 만들어집니다.
    for filename in os.listdir(label_dir):
        if filename.endswith('.json'):
        # 디렉토리의 모든 파일을 대상으로 반복 작업을 수행하는데 이때 조건문을 통해 .json 파일만을 처리하도록 제한합니다.
            image_id = extract_image_id(filename)
             # JSON 파일의 이미지 ID를 추출하여 저장합니다.
            if image_id in image_ids:
            # 조건문을 통해 image_ids 목록에 포함된 경우만 이후 처리를 진행합니다
            # 이는 유효한 이미지 ID에 대한 정보를 수집하기 위해서입니다.
                json_path = os.path.join(label_dir, filename)
                # 현재 디렉토리 경로(label_dir)와 파일 이름(filename)을 합쳐 JSON 파일의 전체 경로를 만듭니다.
                with open(json_path, 'r', encoding='utf-8') as file:
                # 파일을 읽기모드로 열어서 UTF-8 인코딩으로 문자를 읽습니다.
                    data = json.load(file)
                    # JSON 파일의 내용을 읽어 파이썬 딕셔너리로 변환합니다.
                    respondent_id = data['user']['R_id']
                    style = data['item']['style']
                    img_filename = data['item']['imgName']
                    preference = data['item']['survey']['Q5'] 
                    # data에서 응답자 ID, 스타일 정보, 이미지 파일명, 응답자의 스타일 선호도롤 가져와 변수에 저장합니다.
                    # preference 변수는 선호도 값을 저장합니다.(1은 비선호, 2는 선호)
                    if preference == 1:
                        preference_data[respondent_id]['비선호'].append(img_filename)
                        # preference 값이 1이라면 '비선호'리스트에 img_filename을 추가하여 저장합니다.
                    elif preference == 2:
                        preference_data[respondent_id]['선호'].append(img_filename)
                        # preference 값이 2이라면 '선호'리스트에 img_filename을 추가하여 저장합니다.
                        # 이 과정을 통해 응답자 ID별로 선호 및 비선호 이미지 리스트를 구축할 수 있습니다.

    return preference_data
    # 리턴값은 응답자ID를 키로 하고, 각 응답자의 선호와 비선호 이미지를 리스트 형태로 저장한 딕셔너리입니다.
    # 예를 들어 preference_data['respondent_123']는 {'선호': [img1, img2], '비선호': [img3]}와 같은 구조를 가집니다.

In [ ]:
training_preference_data = collect_preference_data(training_image_ids, training_label_dir)
# 함수를 호출하여 training data에 대한 선호 및 비선호 정보를 수집합니다.
validation_preference_data = collect_preference_data(validation_image_ids, validation_label_dir)
# # 함수를 호출하여 validation data에 대한 선호 및 비선호 정보를 수집합니다.

In [ ]:
# Pandas 출력 옵션을 설정합니다. (더 많은 행과 열을 출력하도록 설정)
pd.set_option('display.max_rows', None)  
# max_rows 설정을 사용하여 최대 출력 행 수를 지정합니다.
# None을 지정하면 모든 행 출력하여 한번에 볼 수 있도록 합니다.
pd.set_option('display.max_columns', None)
# max_columns 설정을 사용하여 최대 출력 열 수를 지정합니다.
# None을 지정하면 모든 열을 출력하도록 합니다.
pd.set_option('display.width', 1000)
# width 설정을 사용하여 출력의 가로 폭을 조정합니다.
# 1000으로 설정하면 출력의 가로 너비를 1000문자로 늘려 많은 열을 가진 데이터프레임을 출력하기에 용이하도록 합니다.

In [ ]:
# 열의 최대 너비 설정 (문자열을 일정한 길이로 자르고 공백을 추가)
def format_string(value, max_len=35):
# 매개변수는 포맷할 문자열(value)와 문자열의 최대길이(max_len 기본값은 35)입니다.
    if len(value) > max_len:
        return value[:max_len-3] + '...'  
    # 문자열의 길이가 max_len을 초과하면 max_len-3의 길이로 자른 후 '...'을 추가하여 잘린 부분을 표시합니다.
    else:
        return value.ljust(max_len)
    # 문자열의 길이가 max_len 이하인 경우 문자열의 오른쪽에 공백을 추가해 max_len만큼의 길이로 만듭니다.
    # 이러한 과정을 통해 정렬된 형태로 출력할 수 있습니다.

In [ ]:
# 각 응답자에 대한 스타일 선호/비선호 파일명을 셀에 나누어 표시하는 함수
def prepare_rows_for_respondent(respondent_id, training_preferences, validation_preferences):
# 매개변수는 응답자의 고유ID, 응답자의 training data, validation data에 대한 선호/비선호 정보입니다.
    max_len = max(len(training_preferences['선호']), len(training_preferences['비선호']),
                  len(validation_preferences['선호']), len(validation_preferences['비선호']))
    # 데이터를 처리하기 위해 필요한 행을 결정하기 위해 가장 긴 리스트의 길이를 max_len에 저장합니다.
    rows = []
    for i in range(max_len):
    # 최대 길이만큼 반복하여 각 응답자의 선호/비선호 정보를 행 단위로 처리합니다.
        row = [
            respondent_id if i == 0 else '', 
            # 첫 번째 행에는 응답자 ID를 출력하고, 그 이후 행에는 빈 문자열을 추가합니다.
            format_string(training_preferences['선호'][i]) if i < len(training_preferences['선호']) else format_string(''), 
            # training data의 선호 항목을 가져오고, 해당 인덱스가 유효하지 않으면 빈 문자열을 추가합니다.
            format_string(training_preferences['비선호'][i]) if i < len(training_preferences['비선호']) else format_string(''),  
            # training data의 비선호 항목을 가져오고, 해당 인덱스가 유효하지 않으면 빈 문자열을 추가합니다.
            format_string(validation_preferences['선호'][i]) if i < len(validation_preferences['선호']) else format_string(''),  
            # validation data의 선호 항목을 가져오고, 해당 인덱스가 유효하지 않으면 빈 문자열을 추가합니다.
            format_string(validation_preferences['비선호'][i]) if i < len(validation_preferences['비선호']) else format_string('')  
            # validation data의 비선호 항목을 가져오고, 해당 인덱스가 유효하지 않으면 빈 문자열을 추가합니다.
        ]
        rows.append(row)
        # row를 rows 리스트에 추가합니다.

    return rows

In [ ]:
# 표 경계를 위한 함수입니다.
def print_row_separator():
    print("+------------+-------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+")

In [ ]:
# 상위 100명에 대해 Training과 Validation 스타일 선호/비선호 정보를 나누어 표 작성
def generate_combined_preference_table_separated(training_data, validation_data, top_respondents):
    rows = []  
    # 결과를 저장할 빈 리스트를 초기화합니다.

    # 상위 응답자 ID 목록을 순회
    for respondent_id in top_respondents:
        # 응답자 ID에 해당하는 training 및 validation 스타일 정보를 가져옵니다. 
        # 데이터가 없는 경우 빈 리스트를 사용합니다.
        training_preferences = training_data.get(respondent_id, {'선호': [], '비선호': []})
        validation_preferences = validation_data.get(respondent_id, {'선호': [], '비선호': []})

        rows += prepare_rows_for_respondent(respondent_id, training_preferences, validation_preferences)
        # 응답자별로 스타일 정보를 여러 행에 나누어 추가하기 위해 함수를 호출합니다.

    df = pd.DataFrame(rows, columns=['응답자 ID', 'Training 스타일 선호', 'Training 스타일 비선호', 'Validation 스타일 선호', 'Validation 스타일 비선호'])
    # 수집된 데이터를 바탕으로 Pandas Data Frame을 만들고, 각 열의 이름을 지정하여 Data Frame을 생성합니다.

    print("\n--- 상위 100명에 대한 Training 및 Validation 스타일 선호/비선호 통계 ---\n")
    print_row_separator() # 표의 구분선을 출력합니다.

    print(f"| {'응답자 ID':^7} |{'Training 스타일 선호':^32} | {'Training 스타일 비선호':^30} | {'Validation 스타일 선호':^31} | {'Validation 스타일 비선호':^30} |")
    print_row_separator()  # 다시 구분선을 출력합니다.

    previous_respondent = None  
    # ID를 초기화하여 어떤 응답자도 설정되지 않은 상태를 만듭니다.
    for idx, row in df.iterrows():
    # DataFrame(df)의 각 행을 반복합니다.
        if row['응답자 ID'] != previous_respondent:
        # 응답자 ID가 바뀔 때(현재 ID =! 이전 ID) 구분선을 출력합니다.
            if previous_respondent is not None:  # 첫 번째 행에서는 구분선을 출력하지 않도록 합니다.
                print_row_separator()  

        respondent_id = row['응답자 ID'] if row['응답자 ID'] != previous_respondent else ""
        # 현재 응답자의 ID가 이전 응답자와 다르면 그ID를 respondent_id에 저장하고 아니면 빈 문자열로 설정합니다.
        # 이렇게 하는 이유는 같은 응답자에 대한 중복 ID출력을 피하기 위해서 입니다.
        print(f"| {respondent_id:^10} | {row['Training 스타일 선호']:<35} | {row['Training 스타일 비선호']:<35} | {row['Validation 스타일 선호']:<35} | {row['Validation 스타일 비선호']:<35} |")
        # 각 응답자의 ID와 Training, Validation 스타일 선호 및 비선호 정보를 정렬된 형식으로 출력합니다.

        previous_respondent = row['응답자 ID'] 
        # 현재 응답자의 ID를 이전 응답자로 업데이트합니다.
    print_row_separator()
    # 모든 행을 출력한 후 마지막 구분선을 출력하여 표의 끝을 나타냅니다.

    return df 

In [ ]:
# Training과 Validation에서 공통된 응답자ID 찾기
common_respondents = set(training_preference_data.keys()) & set(validation_preference_data.keys())

In [ ]:
# 응답자별 총 응답 수를 계산하기 위한 딕셔너리 생성
respondent_response_count = {
    # 각 응답자의 ID를 키로 사용하고, 그에 대한 총 응답 수를 값으로 저장합니다.
    respondent_id: (
        len(training_preference_data[respondent_id]['선호']) +  # 해당 응답자의 Training 데이터에서 선호 응답 수
        len(training_preference_data[respondent_id]['비선호']) +  # 해당 응답자의 Training 데이터에서 비선호 응답 수
        len(validation_preference_data[respondent_id]['선호']) +  # 해당 응답자의 Validation 데이터에서 선호 응답 수
        len(validation_preference_data[respondent_id]['비선호'])  # 해당 응답자의 Validation 데이터에서 비선호 응답 수
    )
    # common_respondents 리스트에 있는 각 응답자 ID에 대해 반복합니다.
    for respondent_id in common_respondents
}
# 이 코드는 각 응답자의 총 응답 수를 계산하여 딕셔너리에 저장함으로써,
# 이후에 총 응답 수가 많은 상위 100명을 쉽게 선택할 수 있도록 도와줍니다.


In [ ]:

# 총 응답수 기준 상위 100명의 응답자 선택
top_100_respondents = sorted(respondent_response_count, key=respondent_response_count.get, reverse=True)[:100]

# 표 작성 (상위 100명의 응답자에 대한 데이터)
top_100_preference_df = generate_combined_preference_table_separated(training_preference_data, validation_preference_data, top_100_respondents)
